## MNIST CNN

TODO :
1. [Create validation and sample sets](#Create-validation-and-sample-sets)
2. [Rearrange image files into new directories](#Rearrange-image-files-into-new-directories)
3. [Fine-tuning](#Fine-tuning)
4. [Training](#Training)

In [16]:
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split

from keras.models import Sequential
from keras.layers import  Conv2D, Dense, Dropout, Flatten, Lambda, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
#from keras import backend as K

#K.set_image_dim_ordering('th')

In [2]:
train = pd.read_csv('./data/mnist/train.csv')

In [3]:
X_train = train.drop('label', axis=1)
y_train = train.label

In [4]:
X_test = pd.read_csv('./data/mnist/test.csv')

In [5]:
del train

In [6]:
X_train = X_train.values.reshape(-1, 1, 28, 28)
X_test = X_test.values.reshape(-1, 1, 28, 28)

In [7]:
X_train.shape

(42000, 1, 28, 28)

In [8]:
random_seed = 2

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=random_seed)

In [10]:
def onehot(y, num_classes=None):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
    # Returns
        A binary matrix representation of the input.
    """
    y = np.array(y, dtype='int').ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
    return categorical

In [11]:
Y_train = onehot(y_train, num_classes=10)
Y_val = onehot(y_val, num_classes=10)

In [12]:
mean_x = X_train.mean().astype(np.float32)
std_x = X_train.std().astype(np.float32)

In [13]:
def norm_input(x): return (x - mean_x) / std_x

In [14]:
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1, 28, 28)),
        Conv2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Conv2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(axis=1),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [17]:
model = get_model()

In [18]:
gen = ImageDataGenerator(
    rotation_range=12, 
    width_shift_range=0.1, 
    shear_range=0.3,
    height_shift_range=0.1, 
    zoom_range=0.1)

In [19]:
batches = gen.flow(X_train, Y_train, batch_size=64)
val_batches = gen.flow(X_val, Y_val, batch_size=64)

In [20]:
model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
28140/28140 [==============================] - 136s - loss: 0.2885 - acc: 0.9085 - val_loss: 0.3792 - val_acc: 0.8787


In [22]:
model.save_weights('data/mnist/ft1.h5')

In [23]:
model.load_weights('data/mnist/ft1.h5')

In [21]:
model.optimizer.lr = 0.01

In [22]:
model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
28140/28140 [==============================] - 136s - loss: 0.1177 - acc: 0.9632 - val_loss: 0.1444 - val_acc: 0.9532


In [26]:
model.save_weights('data/mnist/ft2.h5')

In [27]:
model.load_weights('data/mnist/ft2.h5')

In [28]:
model.optimizer.lr = 0.001

In [29]:
model.fit_generator(batches, batches.N, nb_epoch=10, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
28140/28140 [==============================] - 89s - loss: 0.0326 - acc: 0.9898 - val_loss: 0.0414 - val_acc: 0.9872


In [30]:
model.save_weights('data/mnist/ft3.h5')

In [31]:
model.load_weights('data/mnist/ft3.h5')

In [32]:
model.optimizer.lr = 0.0001

In [33]:
model.fit_generator(batches, batches.N, nb_epoch=10, validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/1
28140/28140 [==============================] - 89s - loss: 0.0242 - acc: 0.9926 - val_loss: 0.0389 - val_acc: 0.9889


In [34]:
model.save_weights('data/mnist/ft4.h5')

In [35]:
model.load_weights('data/mnist/ft4.h5')

In [46]:
predictions = model.predict(X_test, batch_size=256)

In [56]:
imageIds = np.arange(1, len(labels) + 1)
imageIds.shape

(28000,)

In [55]:
# select the index  with the max probability
labels = np.argmax(predictions, axis=1)
labels.shape

(28000,)

In [59]:
subm = np.stack([imageIds, labels], axis=1)
subm[:5]

array([[1, 2],
       [2, 0],
       [3, 9],
       [4, 0],
       [5, 3]])

In [60]:
subm_filename = 'subm.csv'

In [61]:
np.savetxt(subm_filename, subm, fmt='%d,%d', header='ImageId,Label', comments='')

In [62]:
from IPython.display import FileLink
FileLink(subm_filename)

/home/ubuntu/nbs/deep-learning/subm.csv